In [ ]:
from mapbox import Static
from IPython.display import Image
import json
import pandas as pd
import seaborn as sns

#city1, region1, city2, region2 = 'sanfrancisco', 20, 'newyork', 56
#city1, region1, city2, region2 = 'sanfrancisco', 48, 'newyork', 34
#city1, region1, city2, region2 = 'sanfrancisco', 48, 'rome', 33
city1, region1, city2, region2 = 'london', 6, 'prague', 2

In [ ]:
mpl.style.use('ggplot')
sns.set('talk', 'whitegrid', 'Set1', font_scale=1.6, rc={'figure.figsize': (8, 12)})
mpl.rcParams['text.usetex'] = True
mpl.rc('font',**{'family':'serif','serif':['Palatino']})

short_to_full = {'amsterdam': 'Amsterdam', 'ankara': 'Ankara', 'antalya': 'Antalya', 'atlanta': 'Atlanta',
 'barcelona': 'Barcelona', 'belohorizonte': 'Belo Horizonte', 'berlin': 'Berlin', 'chicago': 'Chicago',
 'fortaleza': 'Fortaleza', 'helsinki': 'Helsinki', 'houston': 'Houston', 'indianapolis': 'Indianapolis',
 'istanbul': 'İstanbul', 'izmir': 'İzmir', 'kualalumpur': 'Kuala Lumpur', 'london': 'London',
 'losangeles': 'Los Angeles', 'mexicocity': 'Mexico City', 'moscow': 'Moscow', 'newyork': 'New York',
 'paris': 'Paris', 'portoalegre': 'Porto Alegre', 'prague': 'Prague', 'recife': 'Recife',
 'riodejaneiro': 'Rio de Janeiro', 'rome': 'Rome', 'salvador': 'Salvador', 'sandiego': 'San Diego',
 'sanfrancisco': 'San Francisco', 'santiagodechile': 'Santiago de Chile', 'saopaulo': 'São Paulo', 'seattle': 'Seattle',
 'seoul': 'Seoul', 'singapore': 'Singapore', 'stlouis': 'Saint Louis', 'stockholm': 'Stockholm',
 'stpetersburg': 'Saint Petersburg', 'tokyo': 'Tokyo', 'washington': 'Washington', 'yokohama': 'Yokohama'}

# See https://github.com/mapbox/simplestyle-spec/tree/master/1.1.0 for more infos and markers
default_style = {"stroke": "#555555",
            "stroke-opacity": 1.0,
            "stroke-width": 1,
            "fill": "#2196f3",
            "fill-opacity": 0.5,}

cats_name = ['Entertainment', 'Education', 'Food', 'Nightlife', 'Outdoors', 'Shop and Service', 
              'Professional', 'Residence', 'Transport']

fc1, fc2 = short_to_full[city1], short_to_full[city2]
color1, color2 = sns.palettes.color_palette('Set1')[:2]
filename1  = '{}_{}.png'.format(city1, region1)
filename2  = '{}_{}.png'.format(city2, region2)

service = Static(access_token=None)

In [ ]:
regions = {}
for city in (city1, city2):
    with open('regions/{}_distrib.json'.format(city)) as f:
        reg = json.load(f)
    regions[city] = {int(r['properties']['name'].split(' ')[-1]): r for r in reg['features']}

In [ ]:
col1 = '{}, region {}'.format(fc1, region1)
col2 = '{}, region {}'.format(fc2, region2)
df=pd.DataFrame({col1: 100*np.array(regions[city1][region1]['properties']['category_distrib']),
                 col2: 100*np.array(regions[city2][region2]['properties']['category_distrib'])},
             cats_name,)[[col1, col2]]
df

Is there something better available out of the box: http://pandas.pydata.org/pandas-docs/stable/visualization.html#visualization-barplot

In [ ]:
ax=df.plot.barh()
ax.set_xlabel('Percentage of venues')
ax.set_ylabel('Category')
plot_filename = '{}_{}_{}_{}.pdf'.format(city1, region1, city2, region2)
plt.savefig(plot_filename, bbox_inches='tight', pad_inches=0)

In [ ]:
for city, region, color in [(city1, region1, color1), (city2, region2, color2)]:
    filename = '{}_{}.png'.format(city, region)
    feat = regions[city][region]
    feat['properties'].update(default_style)
    feat['properties']['fill'] = mpl.colors.rgb2hex(color)
    response = service.image('mapbox.streets', features=[feat], width=400, height=600)
    with open(filename, 'wb') as output:
        output.write(response.content)

In [ ]:
Image(filename1)

In [ ]:
Image(filename2)

In [ ]:
template="""\\begin{figure*}[tb]
  \centering
    \subfigure[RR1]{
      \includegraphics[width=0.3\linewidth]
      {figures/similar/FF1}
    }
    \subfigure[Venues distribution]{
      \includegraphics[width=0.3\linewidth]
      {figures/similar/FFC}
    }
    \subfigure[RR2]{
      \includegraphics[width=0.3\linewidth]
      {figures/similar/FF2}
    }
	\caption{. \label{fig:casestudy}}
\end{figure*}"""

In [ ]:
print((template.replace('FF1', filename1)
       .replace('RR1', 'Region {}'.format(region1))
       .replace('RR2', 'Region {}'.format(region2))
       .replace('FF2', filename2)
       .replace('FFC', '{}_{}_{}_{}.pdf'.format(city1, region1, city2, region2))))